In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px  
from mongodb_crud import MongoDBCRUD  


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 32218
database_name = "AAC"
collection_name = "animals"

# Initialize MongoDB connection
mongo_crud = MongoDBCRUD(username, password, host, port, database_name)

# Fetch data from MongoDB and convert to pandas DataFrame
documents = mongo_crud.read(collection_name, {})
if isinstance(documents, list):
    df = pd.DataFrame(documents)
    df.drop(columns=['_id'], inplace=True, errors='ignore')
else:
    df = pd.DataFrame()

#########################
# Dashboard Layout / View
#########################
# Initialize the Dash app
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Define the layout of the app
app.layout = html.Div([
    #FIX ME Add in Grazioso Salvare’s logo
    # Display an image (logo) and the titles
    html.Center(html.Img(src='https://i.postimg.cc/Kc1q60sv/Grazioso-Salvare-Logo.png', style={'height': '500px', 'margin-bottom': '10px'})),
    #FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Janncy Mota'))),
    html.Hr(),
    # Dropdown for filtering data
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.Dropdown(
        id='filter-type',
        options=[



            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster/Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='All',
        clearable=False,
    ),
    html.Hr(),
    # Data table for displaying fetched data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    # Divs for graphs and maps
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ])
])

# Callback to update the data table based on the selected filter type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Logic to filter data based on the selection
    if filter_type == 'All' or filter_type == 'Reset':
        query = {}
    elif filter_type in ['Dog', 'Cat']:
        query = {'animal_type': filter_type}
    else:
        # Mapping filters to MongoDB queries
        filter_map = {
            'Water Rescue': {'$or': [{'breed': 'Labrador Retriever Mix'}, {'breed': 'Chesapeake Bay Retriever'}, {'breed': 'Newfoundland'}], 'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}},
            'Mountain/Wilderness Rescue': {'$or': [{'breed': 'German Shepherd'}, {'breed': 'Alaskan Malamute'}, {'breed': 'Old English Sheepdog'}, {'breed': 'Siberian Husky'}, {'breed': 'Rottweiler'}], 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}},
            'Disaster/Individual Tracking': {'$or': [{'breed': 'Doberman Pinscher'}, {'breed': 'German Shepherd'}, {'breed': 'Golden Retriever'}, {'breed': 'Bloodhound'}, {'breed': 'Rottweiler'}], 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}},
        }
        query = filter_map.get(filter_type, {})
    
    documents = mongo_crud.read(collection_name, query)
    if isinstance(documents, list):
        df_filtered = pd.DataFrame(documents)
        df_filtered.drop(columns=['_id'], inplace=True, errors='ignore')
        return df_filtered.to_dict('records')
    else:
        return []

# Callback to update graphs based on the data table content
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Generating pie chart for the available data
    if viewData is None or not viewData:
        return dcc.Graph(figure={'data': []})
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='animal_type', title='Animal Types')
    return dcc.Graph(figure=fig)

# Callback to update map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, derived_virtual_selected_rows):
    # Displaying location on map based on selected data
    if viewData is None or not viewData or derived_virtual_selected_rows is None or not derived_virtual_selected_rows:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip("No data selected"),
                dl.Popup("Select a row in the data table to see the location")
            ])
        ])
    
    selected_row_index = derived_virtual_selected_rows[0]
    dff = pd.DataFrame.from_dict(viewData)
    selected_data = dff.iloc[selected_row_index]
    
    # Using coordinates for map marker
    lat = selected_data.get('latitude', 30.75)
    lon = selected_data.get('longitude', -97.48)
    animal_name = selected_data.get('name', 'Unknown')
    animal_breed = selected_data.get('breed', 'Unknown breed')

    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(f"{animal_name} - {animal_breed}"),
            dl.Popup([
                html.H4(animal_name),
                html.P(animal_breed)
            ])
        ])
    ])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8010)



Dash app running on http://127.0.0.1:8010/
